# **Bank Customer Churn Prediction**

# **1. Introduction**

## 1.1. **About dataset:**
The bank customer churn dataset is a commonly used dataset for predicting customer churn in the banking industry. It contains information on bank customers who either left the bank or continue to be a customer.

dataset: [Binary Classification with a Bank Churn Dataset | Kaggle](https://www.kaggle.com/competitions/playground-series-s4e1/data)

## 1.2. **Columns Description:**
Customer ID: A unique identifier for each customer

- Surname: The customer's surname or last name

- Geography: The country where the customer resides (France, Spain or Germany)

- Credit Score: A numerical value representing the customer's credit score

- Gender: The customer's gender (Male or Female)

- Age: The customer's age.

- Tenure: The number of years the customer has been with the bank

- Balance: The customer's account balance

- NumOfProducts: The number of bank products the customer uses (e.g., savings account, credit card)

- HasCrCard: Whether the customer has a credit card (1 = yes, 0 = no)

- IsActiveMember: Whether the customer is an active member (1 = yes, 0 = no)

- EstimatedSalary: The estimated salary of the customer

- Exited: Whether the customer has churned (1 = yes, 0 = no)

# **2. Import Libary**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go
from scipy.stats import norm
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency,ttest_ind
from IPython.display import display
from plotly.subplots import make_subplots

# **3. Import Dataset**

In [ ]:
# load dataset
path="/content/drive/MyDrive/Projects/data visua/dataset/train.csv"
df = pd.read_csv(path)
df.head(5)

In [ ]:
df.shape

In [ ]:
df.describe()

# **4. Quick Remove Null and Duplicate Value**

In [ ]:
# Remove null values
print(f'Num of Null Values: {df.isnull().sum()}')
df.dropna(inplace=True)

# Remove duplicate values
print(f"Number of Duplicate values: {df.duplicated().sum()}")
df.drop_duplicates(inplace=True)

df.reset_index(drop=True, inplace=True)

# **5. Exploratory Data Analysis (EDA)**

## **5.1. The test of independence - Chi-Square (χ2)**

The test of independence, which asks a question of relationship, such as, “Is there a relationship between gender and churn?”

<img src="https://prwatech.in/blog/wp-content/uploads/2019/07/chisqr.jpg" alt="My Image" width=500>

**Degree of freedom**:
  - df = n - 1 where n is sample size.

**Expected Value** = (row sum * column sum) / table sum

**Statistical Hypothesis:**

  - H0 (Original Hypothesis): The two variables are independent.

  - H1 (Alternative Hypothesis): The two variables are not independent.

If O and E are close to each other → No evidence that the two variables are related → accept H0

If O and E are very different → It is possible that the two variables are related → reject H0

**Ref:**

- Investopedia
  - Chi-Square (χ2) Statistic: What It Is, Examples, How and When to Use the Test: https://www.investopedia.com/terms/c/chi-square-statistic.asp#:~:text=A%20chi-square%20%28%CF%872%29%20statistic%20is%20a%20test%20that,variables%2C%20and%20drawn%20from%20a%20large%20enough%20sample.

- Stat Trek
  - How to Calculate Degrees of Freedom: https://stattrek.com/statistics/degrees-of-freedom
  
- Statology
  - How to Find Expected Counts in Chi-Square Tests: https://www.statology.org/expected-counts-chi-square/

### **5.1.1. Example:**

- H0: Gender and Exited are independent.
- H1: Gender and Exited are not independent.

#### **Expected Value - E**

| Gender   | Exited = 0 | Exited = 1 | total |
| -------- | ------------------ | ------------------- | --------- |
| Female   | 51779             | 20105              | 71884    |
| Male     | 78334             | 14816              | 93150    |
| **Total** | 130113            | 34921              | 165034   |


 - Female (F0):

  - F0 = (71884 x 130113) / 165034 = 56674.4302

=> We get:

| Gender | E(Stay)  | E(Churn) |
| ------ | -------- | -------- |
| Female | 56673.43 | 15210.57 |
| Male   | 73439.57 | 19710.43 |

#### **Chi-Square (χ2):**

- χ2(F0) = (51779 - 56673.4)² / 56673.4 ≈ 422.69

| Gender | E(Stay)  | E(Churn) |
| ------ | -------- | -------- |
| Female |422.69 |1574.92  |
| Male   | 326.19 | 1215.37 |

- χ2 = 422.69 + 1574.92 + 326.19 + 1215.37 =3539.17

Degree of freedom (n = 2):  
  - 2 - 1 = 1

Alpha (α = 0.05)

 Z-crit = 3.841

 3539.17 > 3.841

In [ ]:
# df.dtypes

In [ ]:
def chi_square_test_all(df, target="Exited"):
    results = []
    categorical_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
    categorical_cols += [col for col in df.columns if df[col].nunique() < 10 and col != target]

    # Remove duplicates
    categorical_cols = list(dict.fromkeys(categorical_cols))

    for col in categorical_cols:
        contingency = pd.crosstab(df[col], df[target])
        chi2, p, dof, _ = chi2_contingency(contingency)
        results.append((col, round(chi2, 2), round(p, 4), dof))

    result_df = pd.DataFrame(results, columns=["Feature", "Chi2", "p-value", "Degrees of Freedom"])
    return result_df.sort_values("p-value")

chi_square_test_all(df)

- NumOfProducts and IsActiveMember are the most impactful predictors of customer churn.

- Geography and Gender are also relevant and worth including in a predictive model.

- Surname should be excluded or carefully encoded due to its high cardinality and low interpretability.

- HasCrCard has limited predictive power but can still contribute when combined with other variables.

## **5.2. Remove Surname**

Reason for deletion:

1. They may be repeated and are not guaranteed to identify a specific person and may be subject to data noise.
2. That is considered sensitive information. And it is illegal in many countries to use that data.

**Ref:**
- Genealogy Explained
  - Are People With the Same Last Name Related?: https://www.genealogyexplained.com/are-people-with-the-same-last-name-related/

- the GDPR Compliance Consultancy
  - Personal Data vs Sensitive Data – Understanding the Difference: https://thegdprcomplianceconsultancy.co.uk/personal-data-vs-sensitive-data-understanding-the-difference/


In [ ]:
# Count unique Surname
unique_sur = df["Surname"].nunique()
print(f"Total Number of Surname: {df.shape[0]} ")
print(f"Number of unique Surname: {unique_sur}")

In [ ]:
# count each value of Surname
count_values = df["Surname"].value_counts()
count_values

In [ ]:
# min max average num of surname
min_value = count_values.min()
max_value = count_values.max()
avge = count_values.mean()
print(f"Maximum: {max_value}")
print(f"Minimum: {min_value}")
print(f"Average: {avge}")

In [ ]:
# Word cloud
from wordcloud import WordCloud

# Concatenate all surnames into a single string
surnames_text = ' '.join(df['Surname'])

# Generate the word cloud
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = None,
                min_font_size = 10).generate(surnames_text)

# Plot the word cloud
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

## **5.3. Customer Churn Distribution**

In [ ]:
# 5.1. Customer Churn Distribution

#  Exited
churn_counts = df["Exited"].value_counts().reset_index()
churn_counts.columns = ["Exited", "Count"]

# mapping color
map_color_exited = {"0": "deepskyblue", "1": "red"}

# convert Exited to string to mapping color
churn_counts["Exited"] = churn_counts["Exited"].astype(str)

# Pie Charts
fig = px.pie(
    churn_counts,
    names="Exited",
    values="Count",
    color="Exited",
    color_discrete_map=map_color_exited,
    title="Customer Churn Distribution"
)

fig.update_layout(
    width=1000,
    height=650
)

fig.update_traces(textinfo='percent+label+value')
fig.show()

- 130,113 customers (78.8%) stayed with the bank.
- 34,921 customers (21.2%) churned (left the bank).

The churn rate is moderately low (~21%), which means the bank is doing reasonably well in customer retention.

However, nearly 1 in 5 customers leaving still poses a risk, especially considering that acquiring new customers is usually more expensive than retaining current ones.

## **5.4. Gender**

### **5.4.1. Churn Distribution by Gender**

In [ ]:
## 5.2. Churn Distribution by Gender

def plot_churn_by_gender(df, mapcolor):
    # Total and churn
    n_female = df[df["Gender"] == "Female"].shape[0]
    n_male = df[df["Gender"] == "Male"].shape[0]
    total = n_female + n_male

    # filter female, male churn
    female_churn = df[(df["Gender"] == "Female") & (df["Exited"] == 1)].shape[0]
    male_churn = df[(df["Gender"] == "Male") & (df["Exited"] == 1)].shape[0]

    # get percent
    pct_female_total = round((n_female / total) * 100, 2)
    pct_male_total = round((n_male / total) * 100, 2)

    pct_female_churn = round((female_churn / n_female) * 100, 2)
    pct_male_churn = round((male_churn / n_male) * 100, 2)


    map_color = mapcolor

    # plot
    fig = px.histogram(
        df,
        x="Gender",
        color="Exited",
        barmode="group",
        title=f"Churn by Gender | Female: {pct_female_total}% of total, Churn of female: {pct_female_churn}% | Male: {pct_male_total}% of total, Churn of male: {pct_male_churn}%",
        color_discrete_map=map_color,
        category_orders={"Gender": ["Female", "Male"]},
        text_auto=True
    )

    fig.update_layout(
        xaxis_title="Gender",
        yaxis_title="Number of Customers",
        bargap=0.2
    )

    fig.update_traces(textfont_size=14, textposition='outside')
    fig.show()

In [ ]:
map_color_exited = {0: "deepskyblue", 1: "red"}

In [ ]:
plot_churn_by_gender(df, map_color_exited)

There are more male customers than female in the dataset:

- Male: ~93,150 (78,334 stayed, 14,816 churned)

- Female: ~71,884 (51,779 stayed, 20,105 churned)

Churn Rate by Gender:

- Female: ~28%

- Male: ~15.9%

Female customers churn significantly more than male customers.
This may suggest a gap in service satisfaction or product relevance for female users, which the bank should investigate further.

### 5.4.2. One-Tailed Two-Proportion Z-Test

- z = (p1-p2) / √p(1-p)(1/n1+1/n2)

 - p1, p2:  sample proportions of churn for female and male groups
 - n1, n2:  total number of customers in each group
 - p: pooled proportion

- p = (p1n1 + p2n2)/(n1+n2)

**Ref**
- Statology
  - Two Proportion Z-Test: Definition, Formula, and Example: https://www.statology.org/two-proportion-z-test/

#### 5.4.2.1 Female vs Male



The hypothesis:

- Null Hypothesis (H0): p₁ ≤ p₂
( Female churn rate ≤ Male churn rate)

- Alternative Hypothesis (H1):  p₁ > p₂
( Female churn rate > Male churn rate)

| gender   | Churned (x) | 	Total (n) | 	Churn Rate (p) |
|--------|---------------|-------------|-----------------|
| Female     | 20.105        | 71.884      | 27,97%          |
| Male    | 14.816        | 93.150      | 15,91%          |

In [ ]:
# Churned (x),	Total (n) , Churn Rate (p)
x1, n1 = 20105, 71884
x2, n2 = 14816, 93150

p1 = x1 / n1
p2 = x2 / n2
p = (x1 + x2) / (n1 + n2)

z = round((p1 - p2) / np.sqrt(p * (1 - p) * (1/n1 + 1/n2)), 4)
p_value = round(1 - norm.cdf(z), 10)

# Distribution
x_vals = np.linspace(-10, 60, 1000)
y_vals = norm.pdf(x_vals)

fig = go.Figure()

# Normal curve
fig.add_trace(go.Scatter(x=x_vals, y=y_vals, mode='lines', name='Standard Normal Distribution', line=dict(color='blue')))

# Rejection region
fig.add_trace(go.Scatter(
    x=x_vals,
    y=np.where(x_vals >= 1.645, y_vals, None),
    fill='tozeroy',
    mode='none',
    name='Rejection Region (α = 0.05)',
    fillcolor='rgba(255,0,0,0.3)'
))

# Z-score line
fig.add_trace(go.Scatter(
    x=[z, z],
    y=[0, 0.4],  # constant height (just for visualization)
    mode='lines',
    line=dict(color='darkred', dash='dash'),
    name=f'Z = {z}'
))

# Annotation for p-value
fig.add_annotation(
    x=z,
    y=0.03,
    text=f"p-value ≈ {p_value}",
    showarrow=True,
    arrowhead=1,
    ax=-60,
    ay=-30,
    font=dict(color="darkred")
)

# Layout
fig.update_layout(
    title="Interactive One-Tailed Z-Test: Female Churn Rate > Male Churn Rate",
    xaxis_title="Z-score",
    yaxis_title="Probability Density",
    xaxis_range=[ -10, 60],
    showlegend=True,
    width=900,
    height=500
)

fig.show()

### 5.4.3. Conclusion

- The blue curve represents the standard normal distribution (Z-distribution) with mean = 0 and standard deviation = 1.

- The red shaded area starting from Z = 1.645 is the rejection region for a one-tailed test at significance level α = 0.05.

- The dashed red vertical line at Z = 59.491 indicates the actual result of your Z-Test.

- The annotation "p-value ≈ 0.0" highlights that the probability of getting such a high Z-score under the null hypothesis is virtually 0.

## **5.5. Geography**

### 5.5.1 Churn by Geography

In [ ]:
def plot_churn_by_geography(df):
    # all unique regions
    regions = df["Geography"].unique()
    total = df.shape[0]

    # summary for each region
    summaries = []
    for region in regions:
        n_region = df[df["Geography"] == region].shape[0]
        churn_region = df[(df["Geography"] == region) & (df["Exited"] == 1)].shape[0]
        pct_region_total = round((n_region / total) * 100, 2)
        pct_region_churn = round((churn_region / n_region) * 100, 2)
        summaries.append(f"{region}: {pct_region_total}% of total, Churn of {region}: {pct_region_churn}%")

    # title
    title_text = "Churn by Geography | " + " | ".join(summaries)

    # convert Exited to string for color mapping
    df_plot = df.copy()
    df_plot["Exited"] = df_plot["Exited"].astype(str)

    # Plot
    fig = px.histogram(
        df_plot,
        x="Geography",
        color="Exited",
        barmode="group",
        title=title_text,
        color_discrete_map = {'0': "deepskyblue", '1': "red"},
        category_orders={"Geography": sorted(regions)},
        text_auto=True
    )

    fig.update_layout(
        xaxis_title="Geography",
        yaxis_title="Number of Customers",
        bargap=0.2
    )

    fig.update_traces(textfont_size=14, textposition='outside')
    fig.show()


In [ ]:
plot_churn_by_geography(df)

### 5.5.2.  One-Tailed Two-Proportion Z-Test

| Geography   | Churned (x) | 	Total (n) | 	Churn Rate (p) |
|--------|---------------|-------------|-----------------|
| France     | 15,572        | 94,215      | 16.53%         |
| Germany    | 13,114        | 34,606      | 37.90%         |
| Spain    | 6,235       | 36,213     | 17.22%         |

#### 5.5.2.1. Germany vs France
- H0: p_GER ≤ p_FR
- H1: p_GER > p_FR

In [ ]:
# Germany vs France
# churned (X), Total (n)
x1, n1 = 13114, 34606  # Germany
x2, n2 = 15572, 94215  # France

# Proportions
p1 = x1 / n1  # Germany churn rate
p2 = x2 / n2  # France churn rate

# Pooled proportion
p = (x1 + x2) / (n1 + n2)

# Z-score and p-value
z = round((p1 - p2) / np.sqrt(p * (1 - p) * (1/n1 + 1/n2)), 4)
p_value = round(1 - norm.cdf(z), 10)

# Z-distribution plot
x_vals = np.linspace(-10, 90, 1000)
y_vals = norm.pdf(x_vals)

fig = go.Figure()

# Standard normal curve
fig.add_trace(go.Scatter(x=x_vals, y=y_vals, mode='lines',
                         name='Standard Normal Distribution', line=dict(color='blue')))

# Rejection region (α = 0.05, Z = 1.645)
fig.add_trace(go.Scatter(
    x=x_vals,
    y=np.where(x_vals >= 1.645, y_vals, None),
    fill='tozeroy',
    mode='none',
    name='Rejection Region (α = 0.05)',
    fillcolor='rgba(255,0,0,0.3)'
))

# Z-score line
fig.add_trace(go.Scatter(
    x=[z, z],
    y=[0, 0.4],
    mode='lines',
    line=dict(color='darkred', dash='dash'),
    name=f'Z = {z}'
))

# Annotate p-value
fig.add_annotation(
    x=z,
    y=0.03,
    text=f"p-value ≈ {p_value}",
    showarrow=True,
    arrowhead=1,
    ax=-60,
    ay=-30,
    font=dict(color="darkred")
)

# Layout
fig.update_layout(
    title="Z-Test: Germany Churn Rate > France Churn Rate",
    xaxis_title="Z-score",
    yaxis_title="Probability Density",
    xaxis_range=[-10, 90],
    showlegend=True,
    width=900,
    height=500
)

fig.show()

- The Z-score between Germany and France is Z = 81.7043

- The red region is the rejection zone for significance level α = 0.05, starting at Z = 1.645

- Because Z ≫ 1.645 and p-value ≈ 0.0, we reject the null hypothesis H₀

-> Germany has the highest churn rate

#### 5.5.2.1. Germany vs Spain
- H0: p_GER ≤ p_SP

- H1: p_GER > p_SP

In [ ]:
# Germany vs Spain
# churned (X), Total (n)
x1, n1 = 13114, 34606   # Germany
x2, n2 = 6235, 36213    # Spain

# Proportions
p1 = x1 / n1  # Germany churn rate
p2 = x2 / n2  # Spain churn rate

# Pooled proportion
p = (x1 + x2) / (n1 + n2)

# Z-score and p-value
z = round((p1 - p2) / np.sqrt(p * (1 - p) * (1/n1 + 1/n2)), 4)
p_value = round(1 - norm.cdf(z), 10)

# Z-distribution for plot
x_vals = np.linspace(-10, 90, 1000)
y_vals = norm.pdf(x_vals)

# Create figure
fig = go.Figure()

# Normal curve
fig.add_trace(go.Scatter(x=x_vals, y=y_vals, mode='lines', name='Standard Normal Distribution', line=dict(color='blue')))

# Rejection region (α = 0.05 → Z = 1.645)
fig.add_trace(go.Scatter(
    x=x_vals,
    y=np.where(x_vals >= 1.645, y_vals, None),
    fill='tozeroy',
    mode='none',
    name='Rejection Region (α = 0.05)',
    fillcolor='rgba(255,0,0,0.3)'
))

# Z-score line
fig.add_trace(go.Scatter(
    x=[z, z],
    y=[0, 0.4],
    mode='lines',
    line=dict(color='darkred', dash='dash'),
    name=f'Z = {z}'
))

# Annotate p-value
fig.add_annotation(
    x=z,
    y=0.03,
    text=f"p-value ≈ {p_value}",
    showarrow=True,
    arrowhead=1,
    ax=-60,
    ay=-30,
    font=dict(color="darkred")
)

# Layout
fig.update_layout(
    title="Z-Test: Germany Churn Rate > Spain Churn Rate",
    xaxis_title="Z-score",
    yaxis_title="Probability Density",
    xaxis_range=[-10, 90],
    showlegend=True,
    width=900,
    height=500
)

fig.show()

- The chart shows the standard normal distribution (blue), with the rejection region shaded in red starting at Z = 1.645

- The dashed red line at Z = 61.2722 represents the observed Z-score for Germany vs Spain

- The p-value ≈ 0.0 indicates that such an extreme Z-score is virtually impossible under the null hypothesis

- Since Z > 1.645 and p ≈ 0, we reject the null hypothesis

-> Germany has the highest churn rate

### 5.5.3. Conclusion

Germany has a higher churn rate than both France and Spain, with very high statistical significance (Z > 60, p ≈ 0 in both pairs of tests).

## **5.6. Geography and Gender**  

### 5.6.1. Number of Gender by Geography

In [ ]:
# # Group by Geography and Gender
gender_geo_counts = df.groupby(["Geography", "Gender"]).size().reset_index(name="Count")

# # Calculate total and percent ( ALL Dataset)
# total = gender_geo_counts["Count"].sum()
# gender_geo_counts["Percent"] = round((gender_geo_counts["Count"] / total) * 100, 2)

# Calculate total per country and percent
total_per_country = gender_geo_counts.groupby("Geography")["Count"].transform("sum")
gender_geo_counts["Percent"] = round((gender_geo_counts["Count"] / total_per_country) * 100, 2)

# # Create label: "Count (Percent%)"
gender_geo_counts["Label"] = gender_geo_counts["Count"].astype(str) + " (" + gender_geo_counts["Percent"].astype(str) + "%)"

In [ ]:
# Bar Chart
fig = px.bar(
    gender_geo_counts,
    x="Geography",
    y="Count",
    color="Gender",
    barmode="group",
    text=gender_geo_counts["Label"],
    title="Number and Percentage of Male and Female Customers per Country"
)

# Update layout
fig.update_layout(
    yaxis_title="Number of Customers",
    xaxis_title="Country",
    bargap=0.2
)

# Show text above bars
fig.update_traces(textposition="outside", textfont_size=13)

# Display chart
fig.show()

### 5.6.2. Churn by Geography and Gender

In [ ]:
# Group: country, Gender and Churn
grouped_churn = df.groupby(["Geography", "Gender", "Exited"]).size().reset_index(name="Count")

grouped_churn["Exited"] = grouped_churn["Exited"].astype(str)
map_color_exited = {"0": "deepskyblue", "1": "red"}

# bar chart
fig = px.bar(
    grouped_churn,
    x="Geography",
    y="Count",
    color="Exited",
    color_discrete_map=map_color_exited,
    barmode="group",
    facet_col="Gender",
    category_orders={"Exited": ["0", "1"]},
    labels={"Exited": "Churned (1 = Yes, 0 = No)"},
    title="Churn vs Stay by Gender and Country"
)

fig.update_layout(
    yaxis_title="Number of Customers",
    xaxis_title="Country",
    bargap=0.2
)

fig.show()


## **5.7 Age**


# **France**
- France Age structure: https://www.indexmundi.com/france/age_structure.html
- Age Groups France: https://www.spotzi.com/en/data-catalog/datasets/age-groups-france/

# **Germany**
- Germany Age structure: https://www.indexmundi.com/germany/age_structure.html
- Age Groups Germany: https://www.spotzi.com/en/data-catalog/datasets/age-groups-germany/
- Addressing the challenge of “Ageism in Finance” in Germany: https://www.fintech4longevity.com/blog/addressing-the-challenge-of-ageism-in-finance-in-germany-guest-author-cornelia-schwertner-ceo-and-co-founder-at-brygge


# **Spain**
- Spain Age structure: https://www.indexmundi.com/spain/age_structure.html
- Age Groups Spain: https://www.spotzi.com/en/data-catalog/datasets/age-groups-spain/


# Age Categories - **CIA World Factbook**:

- **Children Age**: 0-14 years
- **Early working Age**: 15-24 years
- **Prime working Age**: 25-54 years
- **Mature working Age**: 55-64 years
- **Elderly**: Age 65 years and over

# Age Categories - **Spotzi**

- **Childhood**: Age 0 - 10
- **Adolescence, Teenager**: Age 10 - 20
- **Young Adult**: Age 20 - 40
- **Middle-Aged**: Age 40 - 65
- **Elderly**: Age 65 Years and Older

# Age Categories - **Longevity Academy**

- **Teens**: Age 16 - 24 years
- **Young adult**: Age 25 - 44 years
- **Middle-aged / Near retirement**: Age 45 - 64 years
- **Elderly**: Age 65 - 74 years

# **Final Age Categories**

- **Early working Age**: Ages 18–24  
- **Prime Working Age**: Ages 25–44  
- **Mid–Late Working Age**: Ages 45–64  
- **Elderly**: Ages 65– Older

How Old Do You Have to Be to Open a Bank Account?: https://www.banks.com/articles/banking/how-old-open-account/#:~:text=Adults%20can%20open%20a%20bank%20account%20by%20themselves,to%20make%20money%20and%20use%20it%20for%20purchases.


### 5.7.1. mapping age group

In [ ]:
MinAge = df['Age'].min()
MaxAge = df['Age'].max()
print(f"Min Age: {MinAge}, Max Age: {MaxAge}")

In [ ]:
def map_age_group(age):
    if 18 <= age <= 24: # 18–24
        return "Early working Age"
    elif 25 <= age <= 44: # 25–44
        return "Prime Working Age"
    elif 45 <= age <= 64:# 45–64
        return "Mid–Late Working Age"
    elif 65 <= age: # 65 - Older
        return "Elderly"
df_copy = df.copy()
df_copy["AgeGroup"] = df_copy["Age"].apply(map_age_group)

In [ ]:
# A chi-square test of independence
def chi_square_test(df, column, target="Exited"):
    """
    Performs a chi-square test of independence between a given column and the target column
    and returns the results in a pandas DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame containing the data.
        column (str): The name of the categorical column to test.
        target (str): The name of the target column (default is "Exited").

    Returns:
        pd.DataFrame: A DataFrame containing the chi-square test results.
    """
    contingency_table = pd.crosstab(df[column], df[target])

    chi2, p_value, dof, expected = chi2_contingency(contingency_table)

    results = {
        "Feature": [column],
        "Chi2": [round(chi2, 2)],
        "p-value": [round(p_value, 4)],
        "Degrees of Freedom": [dof],
    }

    result_df = pd.DataFrame(results)
    return result_df

chi_square_test(df_copy, "AgeGroup")

### 5.7.2 churn by age group

In [ ]:
# Custom order
age_order = {
    "Early working Age": 1,
    "Prime Working Age": 2,
    "Mid–Late Working Age": 3,
    "Elderly": 4
}
# Mapping Age color
mapping_age_color ={'Early working Age':'lightcyan',
                    'Prime Working Age':'cyan',
                    'Mid–Late Working Age':'royalblue',
                    'Elderly':'darkblue'}

# Prepare and sort data
age_counts = (
    df_copy["AgeGroup"]
    .value_counts()
    .rename_axis("AgeGroup")
    .reset_index(name="Count")
    .assign(SortOrder=lambda d: d["AgeGroup"].map(age_order))
    .sort_values("SortOrder")
    .drop(columns="SortOrder")
)

# Plot
fig = px.pie(
    age_counts,
    names="AgeGroup",
    values="Count",
    color="AgeGroup",
    color_discrete_map = mapping_age_color ,
    title="Age Group Distribution",
    category_orders={"AgeGroup": list(age_order.keys())}
)
# Styled Pie Chart
fig.update_traces(hoverinfo='label+percent', textinfo='value', textfont_size=20,
                  marker=dict( line=dict(color='#000000', width=2)))

fig.update_layout(
    width=1000,
    height=650
)

fig.update_traces(textinfo="percent+label+value")
fig.show()


In [ ]:
# Count and Percent
age_churn_dist = (
    df_copy.groupby(["AgeGroup", "Exited"])
    .size()
    .reset_index(name="Count")
)

# Percent each age group
percents = age_churn_dist.groupby("AgeGroup")["Count"].transform("sum")
age_churn_dist["Percent"] = round((age_churn_dist["Count"] / percents) * 100, 2)

# convert to string to mapping color
age_churn_dist["Exited"] = age_churn_dist["Exited"].astype(str)

# sort age group
age_order = ["Early working Age", "Prime Working Age", "Mid–Late Working Age", "Elderly"]
color_map = {"0": "deepskyblue", "1": "red"}

# Bar chart
fig = px.bar(
    age_churn_dist,
    x="AgeGroup",
    y="Count",
    color="Exited",
    barmode="group",
    color_discrete_map=color_map,
    category_orders={"AgeGroup": age_order, "Exited": ["0", "1"]},
    custom_data=["Percent"]
)

fig.update_traces(
    texttemplate="%{y} (%{customdata[0]}%)",
    textposition="outside"
)

fig.update_layout(
    title=dict(text='Churn Distribute by Age'),
    xaxis_title="Age Group",
    yaxis_title="Number of Customers",
    bargap=0.2
)

fig.show()

### 5.7.3. One-Tailed Two-Proportion Z-Test

| Age Group   | Churned (x) | 	Total (n) | 	Churn Rate (p) |
|--------|---------------|-------------|-----------------|
| Early working Age    | 504        | 4.986      | 10.11%         |
| Prime Working Age    | 17.852       | 128.372      | 13.91%         |
| Mid–Late Working Age | 16.153      | 29.701     | 54.38%         |
| Elderly              | 412      | 1971     | 20.9%         |

In [ ]:
def z_test_age_plot(x1, n1, comparisons, title_prefix="Z-Test: Mid–Late Working Age vs"):
    """
    Plot multiple Z-tests comparing Mid–Late Working Age with other groups.

    Parameters:
    - x1: Churned of Mid–Late group
    - n1: Total of Mid–Late group
    - comparisons: list of tuples (label, x2, n2)
    """
    for label, x2, n2 in comparisons:
        # Proportions
        p1 = x1 / n1
        p2 = x2 / n2
        p = (x1 + x2) / (n1 + n2)  # pooled

        # Z-score and p-value (one-tailed)
        z = round((p1 - p2) / np.sqrt(p * (1 - p) * (1/n1 + 1/n2)), 4)
        p_value = round(1 - norm.cdf(z), 10)

        # Z-distribution
        x_vals = np.linspace(-10, 160, 1000)
        y_vals = norm.pdf(x_vals)

        fig = go.Figure()

        # Normal curve
        fig.add_trace(go.Scatter(x=x_vals, y=y_vals, mode='lines',
                                 name='Standard Normal Distribution', line=dict(color='blue')))

        # Rejection region
        fig.add_trace(go.Scatter(
            x=x_vals,
            y=np.where(x_vals >= 1.645, y_vals, None),
            fill='tozeroy',
            mode='none',
            name='Rejection Region (α = 0.05)',
            fillcolor='rgba(255,0,0,0.3)'
        ))

        # Z-score line
        fig.add_trace(go.Scatter(
            x=[z, z],
            y=[0, 0.4],
            mode='lines',
            line=dict(color='darkred', dash='dash'),
            name=f'Z = {z}'
        ))

        # Annotate p-value
        fig.add_annotation(
            x=z,
            y=0.03,
            text=f"p-value ≈ {p_value}",
            showarrow=True,
            arrowhead=1,
            ax=-60,
            ay=-30,
            font=dict(color="darkred")
        )

        # Layout
        fig.update_layout(
            title=f"{title_prefix} {label}",
            xaxis_title="Z-score",
            yaxis_title="Probability Density",
            xaxis_range=[-10, 160],
            showlegend=True,
            width=900,
            height=500
        )

        fig.show()

#### 5.7.3.1. Mid–Late Working Age vs Early working Age; Prime Working Age and Elderly

In [ ]:
# Mid–Late Working Age
x_midlate, n_midlate = 16153, 29701

# Age Group
comparisons = [
    ("Early working Age", 504, 4986),
    ("Prime Working Age", 17852, 128372),
    ("Elderly", 412, 1971)
]

# One-Tailed Two-Proportion Z-Test
z_test_age_plot(x_midlate, n_midlate, comparisons)

### 5.7.4. Conclusion

The Mid–Late Working Age group (45–64 years) shows a significantly higher churn rate compared to all other age groups.

## **5.8. Number of product**

In [ ]:
# unique value Number of product
df['NumOfProducts'].unique()

### 5.8.1. churn by number of product and geography

In [ ]:
# count customers churn by number of product and geography
df_filtered = df[df['Exited'] == 1]
new_df = df_filtered.groupby(["Geography","NumOfProducts"])["Exited"].count().reset_index()
new_df = new_df.pivot(index='Geography', columns='NumOfProducts')['Exited'].fillna(0)

# heatmap
fig = px.imshow(
    new_df,
    x=new_df.columns,
    y=new_df.index,
    color_continuous_scale='YlGnBu',
    text_auto=True
    )

fig.update_layout(
    title="Number of Churned Customers by Number of Products and Geography",
    xaxis_title="Number of Products",
    yaxis_title="Geography",
    coloraxis_colorbar=dict(title="Number of Churned Customers"),
    xaxis=dict(tickmode="array", tickvals=[1, 2, 3, 4]),
    width=1000,
    height=650
)
fig.show()

### 5.8.2. Conclusion

For all three countries (France, Germany, and Spain), the number of churned customers decreases as the number of products increases.

## **5.9. CreditScore**

A credit score is a three-digit number (typically 300–850) that reflects a person’s creditworthiness based on their credit history. It helps lenders assess the risk of lending money. Credit scores are calculated using factors like payment history, debt levels, and credit length, with FICO and VantageScore being the most common models. A good score improves chances for loans, low interest rates, housing, and even jobs.

Base on SoFi Learn:

•   300-580: Poor

•   580-669: Fair

•   670-739: Good

•   740-799: Very good

•   800-850: Exceptional or Excellent


**Ref**
- FinanceCharts
  - Credit Score : https://www.financecharts.com/definitions/credit-score#:~:text=A%20credit%20score%20is%20a%20numerical%20representation%20of,risk%20associated%20with%20lending%20money%20to%20a%20person.

- SoFi Learn
  - Guide to Credit Score Ranges: https://www.sofi.com/learn/content/credit-score-ranges/#:~:text=%E2%80%A2%20Credit%20scores%20range%20from%20300%20to%20850%2C,late%202024%20is%20717%2C%20which%20is%20considered%20good.?msockid=3f5958d6bdba62b732a54aafbc7a631b

### 5.9.1. Mapping credit score group

In [ ]:
# unique credit score
min_credit_score = df['CreditScore'].min()
max_credit_score = df['CreditScore'].max()
print(f"Min Credit Score: {min_credit_score}, Max Credit Score: {max_credit_score}")
print(f"Total unique credit score: {len(df['CreditScore'].unique())}")

In [ ]:
def map_credit_score(credit_score):
    if 300 <= credit_score <= 579:  # 300-579
        return "poor"
    elif 580 <= credit_score <= 669: # 580-669
        return "fair"
    elif 670 <= credit_score <= 739: # 670-739
        return "good"
    elif 740 <= credit_score <= 799: # 740-799
        return "Very_good"
    elif 800 <= credit_score <= 850: # 800-850
        return "exceptional_or_excellent"

In [ ]:
# map creditscore_group and The test of independence
df_copy = df.copy()
df_copy["creditscore_group"] = df_copy["CreditScore"].apply(map_credit_score)

chi_square_test(df_copy, "creditscore_group")

### 5.9.2. Heatmap customers churn by credit score group and geography

In [ ]:
# count customers churn by credit score and geography
df_filtered = df_copy[df['Exited'] == 1]
new_df = df_filtered.groupby(["Geography","creditscore_group"])["Exited"].count().reset_index()
new_df = new_df.pivot(index='Geography', columns='creditscore_group')['Exited'].fillna(0)
new_df

# heatmap
# sort columns
credit_score_order = ["poor", "fair", "good", "Very_good", "exceptional_or_excellent"]
new_df = new_df[credit_score_order]

# Heatmap
fig_heat = px.imshow(
    new_df,
    x=credit_score_order,
    y=new_df.index,
    color_continuous_scale='YlGnBu',
    text_auto=True
)

fig_heat.update_layout(
    title="Number of Churned Customers by Credit Score and Geography",
    xaxis_title="Credit Score Group",
    yaxis_title="Geography",
    coloraxis_colorbar=dict(title="Number of Churned Customers"),
    width=1000,
    height=650
)

fig_heat.show()

- All three countries have similar customer attrition rates at the highest Credit Scores (580-669).

- The trend of customer attrition decreases as Credit Score increases.

-> Scredit score is not affected by Geography

### 5.9.3. Total customer by credit score group

In [ ]:
# Total customer by credit score group
credit_score_counts = (
    df_copy["creditscore_group"]
    .value_counts()
    .reset_index(name="Count")
)

# Sort the DataFrame
credit_score_order = ["poor", "fair", "good", "Very_good", "exceptional_or_excellent"]
credit_score_counts['SortOrder'] = credit_score_counts['creditscore_group'].apply(lambda x: credit_score_order.index(x))
credit_score_counts = credit_score_counts.sort_values('SortOrder').drop(columns='SortOrder')

# Pie chart
fig_pie = px.pie(
    credit_score_counts,
    names="creditscore_group",
    values="Count",
    title="Total Customer Distribution by Credit Score Group",
    category_orders={"creditscore_group": credit_score_order}
)

fig_pie.update_traces(textinfo="percent+value")
fig_pie.update_layout(width=1000, height=650)
fig_pie.show()

### 5.9.4. churn by credit score group

In [ ]:
# count and percent of churn by credit score group
credit_churn_dist = (
    df_copy.groupby(["creditscore_group", "Exited"])
    .size()
    .reset_index(name="Count")
)

# Percent each age group
percents = credit_churn_dist.groupby("creditscore_group")["Count"].transform("sum")
credit_churn_dist["Percent"] = round((credit_churn_dist["Count"] / percents) * 100, 2)

# convert to string to mapping color
credit_churn_dist["Exited"] = credit_churn_dist["Exited"].astype(str)

# sort age group
credit_score_order = ["poor", "fair", "good", "Very_good", "exceptional_or_excellent"]
color_map = {"0": "deepskyblue", "1": "red"}

# Bar chart
fig = px.bar(
    credit_churn_dist,
    x="creditscore_group",
    y="Count",
    color="Exited",
    barmode="group",
    color_discrete_map=color_map,
    category_orders={"creditscore_group": credit_score_order, "Exited": ["0", "1"]},
    custom_data=["Percent"]
)

fig.update_traces(
    texttemplate="%{y} (%{customdata[0]}%)",
    textposition="outside"
)

fig.update_layout(
    title=dict(text='Churn Distribute by Credit Score'),
    xaxis_title="Credit Score Group",
    yaxis_title="Number of Customers",
    bargap=0.2
)

fig.show()

### 5.9.5. One-Tailed Two-Proportion Z-Test

| Credit Score Group             | Churned (x) | Total (n) | Churn Rate (p) |
| ------------------------------ | ----------- | --------- | -------------- |
| **poor**                       | 6,553       | 27,860    | 0.2352         |
| **fair**                       | 13,581      | 62,023    | 0.2190         |
| **good**                       | 9,969       | 52,161    | 0.1911         |
| **Very\_good**                 | 3,467       | 16,357    | 0.2120         |
| **exceptional\_or\_excellent** | 1,352       | 6,633     | 0.2038         |


#### 5.9.5.1. credit score: poor vs fair; good; very_good and exceptional_or_excellent

In [ ]:
# poor credit score
x_midlate, n_midlate = 6553, 27860

# credit score Group
comparisons = [
    ("fair", 13581, 62023),
    ("good", 9969, 	62023),
    ("very_good", 9969, 52161),
    ("exceptional_or_excellent",1352, 6633)
]

title = 'credit score: poor vs'

# One-Tailed Two-Proportion Z-Test
z_test_age_plot(x_midlate, n_midlate, comparisons, title)

### 5.9.6. Conclusion

Customers in the "poor" credit score group are more likely to leave than those in other .

## **5.10. HasCrCard**

### 5.10.1. churn by hascrcard

In [ ]:
# count and percent of churn by hascrcard
hascrcard_churn_dist = (
    df_copy.groupby(["HasCrCard", "Exited"])
    .size()
    .reset_index(name="Count")
)

# Percent each group
percents = hascrcard_churn_dist.groupby("HasCrCard")["Count"].transform("sum")
hascrcard_churn_dist["Percent"] = round((hascrcard_churn_dist["Count"] / percents) * 100, 2)

# convert to string for color mapping
hascrcard_churn_dist["Exited"] = hascrcard_churn_dist["Exited"].astype(str)
hascrcard_churn_dist["HasCrCard"] = hascrcard_churn_dist["HasCrCard"].astype(str)


color_map = {"0": "deepskyblue", "1": "red"}

# Bar chart
fig = px.bar(
    hascrcard_churn_dist,
    x="HasCrCard",
    y="Count",
    color="Exited",
    barmode="group",
    color_discrete_map=color_map,
    category_orders={"HasCrCard": ["0", "1"], "Exited": ["0", "1"]},
    custom_data=["Percent"]
)

fig.update_traces(
    texttemplate="%{y} (%{customdata[0]}%)",
    textposition="outside"
)

fig.update_layout(
    title=dict(text='Churn Distribution by Having a Credit Card'),
    xaxis_title="Has Credit Card (1=Yes, 0=No)",
    yaxis_title="Number of Customers",
    bargap=0.2
)

fig.show()

### 5.10.2. One-Tailed Two-Proportion Z-Test

- H0: Churn of Non-HasCrCard ≤  Churn of Non-HasCrCard
- H1: Churn of Non-HasCrCard >  Churn of Non-HasCrCard

| HasCrCard | Churned (Exited=1)  | Total Customers | Churn Rate |
| --------- |  ---------------------- | --------------- | ---------- |
| 0 (No)    | 9,235           | 40,606          | 22.74%     |
| 1 (Yes)   | 25,686        | 124,428         | 20.64%     |


#### 5.10.2.1. HasCrCard and HasntCrCard

In [ ]:
# HasCrCard
x_midlate, n_midlate = 9235, 40606

comparisons = [
    ("hasn't credit card", 25686, 124428),
]

title = "credit card: has credit card vs"

# One-Tailed Two-Proportion Z-Test
z_test_age_plot(x_midlate, n_midlate, comparisons, title)

### 5.10.3. Conclusion
- Customers'churn decisions are higher even if they have a credit card.

## **5.11. IsActiveMember**

### 5.11.1. Churn by IsActiveMember

In [ ]:
# count and percent of churn by IsActiveMember
activemember_churn_dist = (
    df_copy.groupby(["IsActiveMember", "Exited"])
    .size()
    .reset_index(name="Count")
)

# Percent each group
percents = activemember_churn_dist.groupby("IsActiveMember")["Count"].transform("sum")
activemember_churn_dist["Percent"] = round((activemember_churn_dist["Count"] / percents) * 100, 2)

# convert to string for color mapping
activemember_churn_dist["Exited"] = activemember_churn_dist["Exited"].astype(str)
activemember_churn_dist["IsActiveMember"] = activemember_churn_dist["IsActiveMember"].astype(str)


color_map = {"0": "deepskyblue", "1": "red"}

# Bar chart
fig = px.bar(
    activemember_churn_dist,
    x="IsActiveMember",
    y="Count",
    color="Exited",
    barmode="group",
    color_discrete_map=color_map,
    category_orders={"IsActiveMember": ["0", "1"], "Exited": ["0", "1"]},
    custom_data=["Percent"]
)

fig.update_traces(
    texttemplate="%{y} (%{customdata[0]}%)",
    textposition="outside"
)

fig.update_layout(
    title=dict(text='Churn Distribution by Active Member'),
    xaxis_title="IsActiveMember (1=Yes, 0=No)",
    yaxis_title="Number of Customers",
    bargap=0.2
)

fig.show()

- This makes sense as more active customers use than churn.

## **5.12. Churn by   HasCreditCard, IsActiveMember credit score,NumOfProduct,**

In [ ]:
def churn_CreditCard_ActiveMember(FilterCD = 0, FilterAM = 0):
  # Filter HasCreditCard and IsActiveMember
  filtered_df = df[(df["HasCrCard"] == FilterCD) & (df["IsActiveMember"] == FilterAM)].copy()

  # Calculate churn percentage for the title
  total_customers = filtered_df.shape[0]
  churned_customers = filtered_df[filtered_df["Exited"] == 1].shape[0]
  churn_percent = (churned_customers / total_customers) * 100 if total_customers > 0 else 0

  # Split Violin Plot
  fig = go.Figure()

  for exited_value, side, name, color in zip([0, 1], ['negative', 'positive'], ['Not Churned', 'Churned'], ['blue', 'red']):
      # Filter by Exited value within the filtered_df
      exited_df = filtered_df[filtered_df["Exited"] == exited_value]

      # Calculate counts and percentages for each NumOfProducts bin
      counts_by_product = exited_df['NumOfProducts'].value_counts().sort_index()
      total_in_group = counts_by_product.sum()
      percentages_by_product = (counts_by_product / total_in_group * 100).round(2)

      fig.add_trace(go.Violin(
          x=exited_df["NumOfProducts"],
          y=exited_df["Age"],
          legendgroup=name,
          scalegroup=name,
          name=name,
          side=side,
          line_color=color,
          spanmode='hard',
          meanline_visible=True
      ))

      # Add annotations for counts and percentages
      for product, count in counts_by_product.items():
          percent = percentages_by_product[product]
          annotation_text = f"Count: {count}<br>Percent: {percent}%"
          fig.add_annotation(
              x=product + (0.2 if side == 'positive' else -0.2),
              y=exited_df[exited_df['NumOfProducts'] == product]['Age'].mean(),
              text=annotation_text,
              showarrow=False,
              font=dict(
                  size=10,
                  color="black"
              )
          )


  fig.update_layout(
      title=f"Churn by Age vs NumOfProducts [ Customers: HasCrCard = {FilterCD} & IsActiveMember = {FilterAM}] <br>Churn Rate of Filter: {churn_percent:.2f}%",
      xaxis_title="Number of Products",
      yaxis_title="Age",
      violingap=0,
      violinmode='overlay',
      width=1500,
      height=650

  )

  fig.show()

### 5.12.1. HasCreditCard = 0 and IsActiveMember = 0

In [ ]:
# Filter HasCreditCard = 0 and IsActiveMember = 0
churn_CreditCard_ActiveMember(0, 0)

### 5.12.2.  HasCreditCard = 0 and IsActiveMember = 1

In [ ]:
# Filter HasCreditCard = 0 and IsActiveMember = 1
churn_CreditCard_ActiveMember(0, 1)

### 5.12.3.  HasCreditCard =  and IsActiveMember = 1

In [ ]:
# Filter HasCreditCard = 1 and IsActiveMember = 1
churn_CreditCard_ActiveMember(1, 1)

### 5.12.3.  HasCreditCard =  and IsActiveMember = 0

In [ ]:
# Filter HasCreditCard = 1 and IsActiveMember = 0
churn_CreditCard_ActiveMember(1, 0)

### 5.12.4. Conclusion



| HasCrCard | IsActiveMember | Churn Rate (%) | Churn Count | Not Churn Count | Total Customers |
| --------- | -------------- | -------------- | ----------- | --------------- | --------------- |
| 0         | 0              | 30.94%         | 6,079       | 13,567          | 19,646          |
| 0         | 1              | 15.06%         | 3,178       | 17,804          | 20,982          |
| 1         | 1              | 11.67%         | 7,141       | 54,048          | 61,189          |
| 1         | 0              | 29.33%         | 18,545      | 44,694          | 63,239          |



## **Final Conclusion**

Customer Churn Decisions Made Easiest:

1. Gender: Female
2. Geography: Germany
3. Age: 45 - 64
4. Number of product: 1
5. Credit Score: 300-579
6. Has Credit Card: No
7. Is Active Member: 0
